#  RAG 체인 구성
- Naïve RAG 구현

### **학습 목표:**  RAG 기반의 질의응답 시스템을 구현할 수 있다

### **실습 자료**: 

- data/transformer.pdf

---

# 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

False

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

`(3) 문서 로드`

In [3]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 로더 초기화
pdf_loader = PyPDFLoader('./data/transformer.pdf')

# 동기 로딩
pdf_docs = pdf_loader.load()
print(f'PDF 문서 개수: {len(pdf_docs)}')

PDF 문서 개수: 15


`(4) 텍스트 분할`

In [4]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

# Hugging Face의 임베딩 모델 생성
embeddings_huggingface = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# 토크나이저 직접 접근
tokenizer = embeddings_huggingface._client.tokenizer

# 토크나이저를 사용한 예시
text = "테스트 텍스트입니다."
tokens = tokenizer(text)
print(tokens)

# 토크나이저 설정 확인
print(tokenizer.model_max_length)  # 최대 토큰 길이
print(tokenizer.vocab_size)        # 어휘 크기


{'input_ids': [0, 153924, 239355, 5826, 5, 2], 'attention_mask': [1, 1, 1, 1, 1, 1]}
8192
250002


In [5]:
# 토큰 수를 계산하는 함수
def count_tokens(text):
    return len(tokenizer(text)['input_ids'])

# 토큰 수 계산
text = "테스트 텍스트입니다."
print(count_tokens(text))

6


In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 텍스트 분할기 생성
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,                      
    chunk_overlap=100,           
    length_function=count_tokens,         # 토큰 수를 기준으로 분할
    separators=["\n\n", "\n", " ", ""],   # 구분자 - 재귀적으로 순차적으로 적용 
)

# 텍스트 분할
chunks = text_splitter.split_documents(pdf_docs)
print(f"생성된 텍스트 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")
print(f"각 청크의 토큰 수: {list(count_tokens(chunk.page_content) for chunk in chunks)}")

생성된 텍스트 청크 수: 38
각 청크의 길이: [1378, 1796, 1831, 1857, 1292, 1609, 503, 1554, 1278, 1362, 1608, 833, 1418, 1680, 999, 1764, 1604, 539, 1219, 1645, 926, 1213, 1688, 716, 1409, 1626, 624, 1411, 1437, 913, 1493, 1337, 845, 812, 470, 438, 470, 441]
각 청크의 토큰 수: [336, 415, 405, 419, 327, 424, 127, 388, 294, 384, 411, 204, 419, 417, 226, 419, 395, 149, 390, 400, 221, 356, 411, 181, 394, 405, 188, 424, 399, 277, 420, 409, 250, 190, 131, 117, 131, 113]


In [7]:
# 청크의 텍스트 확인
print(chunks[2].page_content)

1 Introduction
Recurrent neural networks, long short-term memory [13] and gated recurrent [7] neural networks
in particular, have been firmly established as state of the art approaches in sequence modeling and
transduction problems such as language modeling and machine translation [ 35, 2, 5]. Numerous
efforts have since continued to push the boundaries of recurrent language models and encoder-decoder
architectures [38, 24, 15].
Recurrent models typically factor computation along the symbol positions of the input and output
sequences. Aligning the positions to steps in computation time, they generate a sequence of hidden
states ht, as a function of the previous hidden state ht−1 and the input for position t. This inherently
sequential nature precludes parallelization within training examples, which becomes critical at longer
sequence lengths, as memory constraints limit batching across examples. Recent work has achieved
significant improvements in computational efficiency through facto

# 벡터 저장소 기반 RAG 검색기 (Retriever)



`(1) 벡터 저장소 초기화`
- chroma 사용
- cosine distance 기준으로 인덱싱 

In [9]:
from langchain_chroma import Chroma

# Chroma 벡터 저장소 생성하기
chroma_db = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings_huggingface,    # huggingface 임베딩 사용
    collection_name="db_transformer",    # 컬렉션 이름
    persist_directory="./chroma_db",
    collection_metadata = {'hnsw:space': 'cosine'}, # l2, ip, cosine 중에서 선택 
)

# 현재 저장된 컬렉션 데이터 확인
chroma_db.get()

{'ids': ['e98fb16f-5c4b-4290-b64a-c3224f1c4581',
  '758c5840-20f7-43cb-9238-d3a6572ec2bd',
  '76cadb0d-0294-4e67-aaae-510f7c16e7af',
  '34b2a815-edeb-4050-990e-9b4b6fcdc7cb',
  '096bbdd4-6d0f-4b07-b07b-ba30182c38c2',
  'a2ca8512-7ac2-48d9-948d-645800439195',
  '5dc7f1ab-444a-4ce4-acf1-006064dbf415',
  '2b3620d9-81d5-44b7-86c7-32a0a27ba34c',
  '7d2850b5-a2cd-4279-b06e-7377b312fc07',
  '2669c4cb-7e8a-4f15-8495-b791012ab047',
  'cd78d9cb-79ab-40e6-a09b-077a2a191b01',
  'cdcd7c9d-56f9-4979-8617-ed49a45c7c94',
  '779a18c5-7a92-4d59-9f4e-e802b2c605f7',
  'ff23ee52-da96-4d25-983b-b4ae6e6789ea',
  '2beff61a-b3f9-40ac-af69-afe76de436cf',
  'c9cf84fc-ac73-47f6-ab0b-915a6aa00fde',
  'e5a4168b-e4c6-4724-ba6c-d36a623d14fc',
  'fb288057-5ce4-4b0e-9420-6bc7d096ab19',
  'c7a3d9e0-8ff1-45af-b779-eaa94d12192a',
  'eb453d12-c1e9-4f2a-907d-06eb1f8493fc',
  'b6dac4de-dcb9-4cf4-a711-d72db73ca46f',
  '68ba8ca4-c7de-450f-97b9-dfdbc8d64fe5',
  '55f9fc8b-fa3b-4214-a90d-45a8319ba443',
  '3397317c-c34a-4bc8-8ae1-

In [11]:
chroma_db._collection.count()

38

`(2) Top K`

In [10]:
chroma_k_retriever = chroma_db.as_retriever(
    search_kwargs={"k": 2},
)

query = "대표적인 시퀀스 모델은 어떤 것들이 있나요?"
retrieved_docs = chroma_k_retriever.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"-{i}-\n{doc.page_content[:100]}...{doc.page_content[-100:]} [출처: {doc.metadata['source']}]")
    print("-" * 100)

쿼리: 대표적인 시퀀스 모델은 어떤 것들이 있나요?
검색 결과:
-1-
1 Introduction
Recurrent neural networks, long short-term memory [13] and gated recurrent [7] neural...he Transformer allows for significantly more parallelization and can reach a new state of the art in [출처: ./data/transformer.pdf]
----------------------------------------------------------------------------------------------------
-2-
In contrast to RNN sequence-to-sequence models [37], the Transformer outperforms the Berkeley-
Parse... 2016.
[2] Dzmitry Bahdanau, Kyunghyun Cho, and Yoshua Bengio. Neural machine translation by jointly [출처: ./data/transformer.pdf]
----------------------------------------------------------------------------------------------------


`(3) 임계값 지정`
- Similarity score threshold (기준 스코어 이상인 문서를 대상으로 추출)

In [12]:
from langchain_community.utils.math import cosine_similarity

chroma_threshold_retriever = chroma_db.as_retriever(
    search_type='similarity_score_threshold',       # cosine 유사도
    search_kwargs={'score_threshold': 0.5, 'k':2},  # 0.5 이상인 문서를 추출
)

query = "대표적인 시퀀스 모델은 어떤 것들이 있나요?"
retrieved_docs = chroma_threshold_retriever.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for i, doc in enumerate(retrieved_docs, 1):
    score = cosine_similarity(
        [embeddings_huggingface.embed_query(query)], 
        [embeddings_huggingface.embed_query(doc.page_content)]
        )[0][0]
    print(f"-{i}-\n{doc.page_content[:100]}...{doc.page_content[-100:]} [유사도: {score}]")
    print("-" * 100)

쿼리: 대표적인 시퀀스 모델은 어떤 것들이 있나요?
검색 결과:
-1-
1 Introduction
Recurrent neural networks, long short-term memory [13] and gated recurrent [7] neural...he Transformer allows for significantly more parallelization and can reach a new state of the art in [유사도: 0.5069070298521535]
----------------------------------------------------------------------------------------------------
-2-
In contrast to RNN sequence-to-sequence models [37], the Transformer outperforms the Berkeley-
Parse... 2016.
[2] Dzmitry Bahdanau, Kyunghyun Cho, and Yoshua Bengio. Neural machine translation by jointly [유사도: 0.5020664684681173]
----------------------------------------------------------------------------------------------------


`(4) MMR(Maximal Marginal Relevance) 검색`

In [13]:
# MMR - 다양성 고려 (lambda_mult 작을수록 더 다양하게 추출)
chroma_mmr = chroma_db.as_retriever(
    search_type='mmr',
    search_kwargs={
        'k': 3,                 # 검색할 문서의 수
        'fetch_k': 8,           # mmr 알고리즘에 전달할 문서의 수 (fetch_k > k)
        'lambda_mult': 0.5,     # 다양성을 고려하는 정도 (1은 최소 다양성, 0은 최대 다양성을 의미. 기본값은 0.5)
        },
)


query = "대표적인 시퀀스 모델은 어떤 것들이 있나요?"
retrieved_docs = chroma_mmr.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for i, doc in enumerate(retrieved_docs, 1):
    score = cosine_similarity(
        [embeddings_huggingface.embed_query(query)], 
        [embeddings_huggingface.embed_query(doc.page_content)]
        )[0][0]
    print(f"-{i}-\n{doc.page_content[:100]}...{doc.page_content[-100:]} [유사도: {score}]")
    print("-" * 100)

쿼리: 대표적인 시퀀스 모델은 어떤 것들이 있나요?
검색 결과:
-1-
1 Introduction
Recurrent neural networks, long short-term memory [13] and gated recurrent [7] neural...he Transformer allows for significantly more parallelization and can reach a new state of the art in [유사도: 0.5069070298521535]
----------------------------------------------------------------------------------------------------
-2-
Table 1: Maximum path lengths, per-layer complexity and minimum number of sequential operations
for ...ng
corresponds to a sinusoid. The wavelengths form a geometric progression from 2π to 10000 · 2π. We [유사도: 0.4791548293733908]
----------------------------------------------------------------------------------------------------
-3-
from our models and present and discuss examples in the appendix. Not only do individual attention
h..., according to the formula:
lrate = d−0.5
model · min(step_num−0.5, step_num · warmup_steps−1.5) (3) [유사도: 0.4709167797240229]
------------------------------------------------------------

`(5) metadata 필터링 검색`

In [14]:
# 메타데이터 확인
chunks[0].metadata

{'producer': 'pdfTeX-1.40.25',
 'creator': 'LaTeX with hyperref',
 'creationdate': '2024-04-10T21:11:43+00:00',
 'author': '',
 'keywords': '',
 'moddate': '2024-04-10T21:11:43+00:00',
 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5',
 'subject': '',
 'title': '',
 'trapped': '/False',
 'source': './data/transformer.pdf',
 'total_pages': 15,
 'page': 0,
 'page_label': '1'}

In [15]:
# 문서 객체의 metadata를 이용한 필터링
chrom_metadata = chroma_db.as_retriever(
    search_kwargs={
        'filter': {'source': './data/transformer.pdf'},
        'k': 5, 
        }
)

query = "대표적인 시퀀스 모델은 어떤 것들이 있나요?"
retrieved_docs = chrom_metadata.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"-{i}-\n{doc.page_content} [출처: {doc.metadata['source']}]")
    print("-" * 100)

쿼리: 대표적인 시퀀스 모델은 어떤 것들이 있나요?
검색 결과:
-1-
1 Introduction
Recurrent neural networks, long short-term memory [13] and gated recurrent [7] neural networks
in particular, have been firmly established as state of the art approaches in sequence modeling and
transduction problems such as language modeling and machine translation [ 35, 2, 5]. Numerous
efforts have since continued to push the boundaries of recurrent language models and encoder-decoder
architectures [38, 24, 15].
Recurrent models typically factor computation along the symbol positions of the input and output
sequences. Aligning the positions to steps in computation time, they generate a sequence of hidden
states ht, as a function of the previous hidden state ht−1 and the input for position t. This inherently
sequential nature precludes parallelization within training examples, which becomes critical at longer
sequence lengths, as memory constraints limit batching across examples. Recent work has achieved
significant improvements i

`(6) page_content 본문 필터링 검색`

In [16]:
# page_content를 이용한 필터링
chroma_content = chroma_db.as_retriever(
    search_kwargs={
        'k': 2,
        'where_document': {'$contains': 'recurrent'},
        }
)

query = "대표적인 시퀀스 모델은 어떤 것들이 있나요?"
retrieved_docs = chroma_content.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"-{i}-\n{doc.page_content} [출처: {doc.metadata['source']}]")
    print("-" * 100)

쿼리: 대표적인 시퀀스 모델은 어떤 것들이 있나요?
검색 결과:
-1-
1 Introduction
Recurrent neural networks, long short-term memory [13] and gated recurrent [7] neural networks
in particular, have been firmly established as state of the art approaches in sequence modeling and
transduction problems such as language modeling and machine translation [ 35, 2, 5]. Numerous
efforts have since continued to push the boundaries of recurrent language models and encoder-decoder
architectures [38, 24, 15].
Recurrent models typically factor computation along the symbol positions of the input and output
sequences. Aligning the positions to steps in computation time, they generate a sequence of hidden
states ht, as a function of the previous hidden state ht−1 and the input for position t. This inherently
sequential nature precludes parallelization within training examples, which becomes critical at longer
sequence lengths, as memory constraints limit batching across examples. Recent work has achieved
significant improvements i

# [실습 프로젝트] Naive RAG 구현 

- 각 단계별 지시사항에 따라 코드를 완성하세요. 
- 제시된 지시사항과 LangChain 문서를 참조하여 시스템을 구성합니다. 

`(1) 벡터 저장소 설정`
- HuggingFace에서 지원하는 BAAI/bge-m3 임베딩 모델을 사용하여 문서를 벡터화
- FAISS DB를 벡터 스토어로 사용 (IndexFlatL2 사용: 유클리드 거리)

In [1]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings  

# Hugging Face의 임베딩 모델 생성
embeddings_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# 임베딩 차원 확인
embedding = embeddings_model.embed_query("Hello world")
print(f"임베딩 차원: {len(embedding)}")


임베딩 차원: 1024


In [2]:
# Ollama 임베딩 모델을 사용한 FAISS 벡터 저장소 생성
import faiss 
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

# FAISS 인덱스 초기화 (유클리드 거리 사용)
dim = 1024  # 임베딩 차원
faiss_index = faiss.IndexFlatL2(len(embeddings_model.embed_query("hello world"))) 

# FAISS 벡터 저장소 생성
faiss_db = FAISS(
    embedding_function=embeddings_model, ###bge-m3 사용
    index=faiss_index,           # 벡터 검색을 위한 데이터 구조를 정의
    docstore=InMemoryDocstore(), # 문서 저장소 객체를 지정 - 문서의 원본 내용과 메타데이터를 보관
    index_to_docstore_id={},     # 인덱스와 문서 간의 연결을 관리 (매핑 딕셔너리)
)

# 저장된 문서의 갯수 확인
print(faiss_db.index.ntotal)

0


In [3]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 로더 초기화
pdf_loader = PyPDFLoader('./data/transformer.pdf')

# 문서 로드
documents = pdf_loader.load()

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
### 텍스트 분할 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)
print(f"생성된 청크 수: {len(chunks)}")

생성된 청크 수: 49


In [5]:
import uuid

# 문서 id 생성
doc_ids = [str(uuid.uuid4()) for _ in range(len(chunks))]

# 문서를 벡터 저장소에 저장
added_doc_ids = faiss_db.add_documents(documents=chunks, ids=doc_ids)

# 벡터 저장소에 저장된 문서를 확인
print(f"{len(added_doc_ids)}개의 문서가 성공적으로 벡터 저장소에 추가되었습니다.")
print(added_doc_ids)

49개의 문서가 성공적으로 벡터 저장소에 추가되었습니다.
['c831716c-421f-47fe-b62d-449b313723c9', '1f511f6b-a7d8-4478-b30d-3381494a262f', '167628d9-5e61-43f5-8943-cecc3d69664b', 'fb8b95b5-69c8-4f23-bd40-966cb632d642', '397144e4-9913-4372-a8d3-29af5fd570f4', 'be0d9a76-df38-4c5a-9887-b10830e8f60d', '7d7757ea-274b-4c02-b038-7cec71091690', 'aee2cbc2-ab65-49e1-b1c8-3873b4118bbb', 'ac1ed6e9-f800-4629-a931-85bfdd7d8035', 'd6abee48-1afb-426c-82c6-968335312851', '880aeb01-36ea-4d3e-8725-f91bd0e65e57', 'fa41275d-acbb-4a67-8af7-e1bdafc14303', '210854aa-5783-41bf-9503-14e696664e55', 'de1cc328-2e80-45de-9943-5bc7ddcc5328', '46ec9d79-7eb2-4614-8491-68098e83edcb', '9397d7f5-d71f-4e5c-ae3e-038169555690', 'a593b451-3336-40c8-840d-cc3711c25ed1', 'f58dd84a-7f78-4779-84b4-ba9fd133e67b', '2d6b4fd2-4d3b-44d0-a2a1-b8e97784a2f3', '781980bc-9f95-4818-9e99-41e938e55a29', '9c00d6e5-2857-4826-8d5a-76a5048ba8e3', '3b93fff2-3285-42d1-aa45-9a5dc14ca469', 'f6396465-8df1-498f-8719-830bb165aa32', '3a27e1b8-13dc-4c57-965d-ddb6e94abe08', '6fe4ee

`(2) 검색기 정의`
- mmr 검색으로 상위 3개 문서 검색하는 Retriever 사용
- 다양성을 높이는 설정을 사용 

In [6]:
# mmr 검색기 생성
faiss_mmr_retriever = faiss_db.as_retriever(
    search_type='mmr',
    search_kwargs={
        'k': 3,                 # 검색할 문서의 수
        'fetch_k': 8,           # mmr 알고리즘에 전달할 문서의 수 (fetch_k > k)
        'lambda_mult': 0.5,     # 다양성을 고려하는 정도 (1은 최소 다양성, 0은 최대 다양성을 의미. 기본값은 0.5)
        },    
)

In [7]:
# 검색 테스트 
query = "대표적인 시퀀스 모델은 어떤 것들이 있나요?"
retrieved_docs = faiss_mmr_retriever.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"-{i}-\n{doc.page_content[:100]}...{doc.page_content[-100:]}")
    print("-" * 100)

쿼리: 대표적인 시퀀스 모델은 어떤 것들이 있나요?
검색 결과:
-1-
1 Introduction
Recurrent neural networks, long short-term memory [13] and gated recurrent [7] neural...ger
sequence lengths, as memory constraints limit batching across examples. Recent work has achieved
----------------------------------------------------------------------------------------------------
-2-
Table 1: Maximum path lengths, per-layer complexity and minimum number of sequential operations
for ...e
bottoms of the encoder and decoder stacks. The positional encodings have the same dimension dmodel
----------------------------------------------------------------------------------------------------
-3-
Our results in Table 4 show that despite the lack of task-specific tuning our model performs sur-
pr...te of the art. In the former task our best
model outperforms even all previously reported ensembles.
----------------------------------------------------------------------------------------------------


`(3) RAG 프롬프트 구성`

- 작성 기준: 
    - LangChain의 ChatPromptTemplate 클래스 사용
    - 변수 처리는 {context}, {question} 형식 사용
    - 답변은 한글로 출력되도록 프롬프트 작성
    
- 아래 템플릿 코드를 기반으로 다음 내용을 참고하여 작성합니다. 

    1. 프롬프트 구성요소:
        - 작업 지침
        - 컨텍스트 영역
        - 질문 영역
        - 답변 형식 가이드

    2. 작업 지침:
        - 컨텍스트 기반 답변 원칙
        - 외부 지식 사용 제한
        - 불확실성 처리 방법
        - 답변 불가능한 경우의 처리 방법

    3. 답변 형식:
        - 핵심 답변 섹션
        - 근거 제시 섹션
        - 추가 설명 섹션 (필요시)

    4. 제약사항 반영:
        - 답변은 사실에 기반해야 함
        - 추측이나 가정을 최소화해야 함
        - 명확한 근거 제시가 필요함
        - 구조화된 형태로 작성되어야 함

In [ ]:
# Prompt 템플릿 (예시)
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context.

[Context]
{context}

[Question] 
{question}

[Answer]
"""

prompt = ChatPromptTemplate.from_template(template)

In [8]:
# Prompt 템플릿 (여기에 작성하세요)
from langchain.prompts import ChatPromptTemplate

template =  """Answer the question based only on the following context.

[Context]
{context}

[Question] 
{question}

[Answer]
"""

prompt = ChatPromptTemplate.from_template(template)

# 템플릿 출력
prompt.pretty_print()

================================ Human Message =================================

Answer the question based only on the following context.

[Context]
{context}

[Question] 
{question}

[Answer]



`(4) RAG 체인 구성`
- LangChain의 LCEL 문법을 사용
- 검색 결과를 프롬프트의 'context'로 전달하고,
- 사용자가 입력한 질문을 그래도 프롬프트의 'question'에 전달
- LLM 설정:
    - ChatOpenAI 사용 ('gpt-4.1-mini' 모델)
    - temperature: 답변의 일관성을 가져가는 설정값을 사용 
    - 기타 필요한 설정 
- 출력 파서: 문자열 부분만 출력되도록 구성 

In [11]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# LLM 설정
llm = ChatOpenAI(
    model="gpt-4.1-mini", 
    temperature=0.7, 
    top_p=0.9,
    presence_penalty=0.3,
    frequency_penalty=0.3,
    )
# 문서 포맷팅
def format_docs(docs):
    return "\n\n".join([f"{doc.page_content}" for doc in docs])

# RAG 체인 생성
rag_chain = (
    {"context": faiss_mmr_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 체인 실행
query = "대표적인 시퀀스 모델은 어떤 것들이 있나요?"
output = rag_chain.invoke(query)

print(f"쿼리: {query}")
print("답변:")
print(output)

쿼리: 대표적인 시퀀스 모델은 어떤 것들이 있나요?
답변:
대표적인 시퀀스 모델로는 순환 신경망(Recurrent Neural Networks, RNN), 장단기 기억(Long Short-Term Memory, LSTM), 게이트 순환 유닛(Gated Recurrent Units, GRU) 등이 있습니다. 특히 LSTM과 GRU는 언어 모델링과 기계 번역과 같은 시퀀스 모델링 및 변환 문제에서 최첨단 접근법으로 확립되어 있습니다.


`(5) Gradio 스트리밍 구현`
- ChatInterface 사용
- `chain.stream()`으로 응답을 청크 단위로 스트리밍

In [12]:
import gradio as gr
from typing import Iterator

# 스트리밍 응답 생성 함수
def get_streaming_response(message: str, history) -> Iterator[str]:
    
    # RAG Chain 실행 및 스트리밍 응답 생성
    response = f"처리 중인 메시지: {message}"
    for chunk in rag_chain.stream(message):
        if isinstance(chunk, str):
            response += chunk
            yield response

# Gradio 인터페이스 설정
demo = gr.ChatInterface(
    fn=get_streaming_response,
    title="스트리밍 챗봇",
    description="입력한 메시지를 한 글자씩 처리하는 챗봇입니다.",
    analytics_enabled=False,  
)

# 실행
demo.launch()

C:\Users\skssk\AppData\Roaming\Python\Python312\site-packages\gradio\chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [13]:
# demo 실행 종료
demo.close()

Closing server running on port: 7860
